<a href="https://colab.research.google.com/github/vothane/II4I/blob/master/II4I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lets go Bruins. All eyes on center.
![alt text](https://https://media1.giphy.com/media/hJjTwuJjjsVgc/giphy.gif)

![](https://media1.giphy.com/media/hJjTwuJjjsVgc/giphy.gif)

![alt text](https://https://media1.giphy.com/media/hJjTwuJjjsVgc/giphy.gif)